# 이 미친 인간들이 과시 데이터를 전부 링크로 퉁치면 어떡해

In [1]:
links = ['https://archive.aks.ac.kr/insp/item.do#view.do?itemId=insp&gubun=form&upPath=02%5E0204%5E020451&dataId=G002%2BAKS%2BKSM-XD.1872.0000-20101008.B043a_065_00633_XXX',
'https://archive.aks.ac.kr/insp/item.do#view.do?itemId=insp&gubun=form&upPath=02%5E0204%5E020451&dataId=G002%2BAKS%2BKSM-XD.1872.0000-20101008.B043a_065_00634_XXX',
'https://archive.aks.ac.kr/insp/item.do#view.do?itemId=insp&gubun=form&upPath=02%5E0204%5E020451&dataId=G002%2BAKS%2BKSM-XD.1873.0000-20101008.B047a_071_00188_XXX']

In [2]:
import requests
from bs4 import BeautifulSoup
import os
import time

def fetch_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0'
    }
    try:
        res = requests.get(url, headers=headers)
        if res.status_code == 200:
            return res.text
        else:
            print(f"Failed to fetch {url}: {res.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

def extract_main_text(html):
    soup = BeautifulSoup(html, 'html.parser')
    # 페이지 구조에 따라 조정 필요
    content = soup.find("div", class_="board-view")  # 예시
    return content.get_text(strip=True) if content else None

def archive_crawl(url_list, output_dir="archive_data"):
    os.makedirs(output_dir, exist_ok=True)
    for i, url in enumerate(url_list):
        html = fetch_page(url)
        if html:
            text = extract_main_text(html)
            if text:
                with open(f"{output_dir}/doc_{i}.txt", "w", encoding="utf-8") as f:
                    f.write(text)
        time.sleep(1.5)  # 서버에 무리 안 가게

# 예시
archive_crawl(links)

/Users/songhune/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
import requests
from bs4 import BeautifulSoup

# 예시 URL (archive.aks.ac.kr의 시권 데이터 중 하나)
headers = {
    'User-Agent': 'Mozilla/5.0'
}

res = requests.get(links[0], headers=headers)
res.raise_for_status()  # 에러 발생 시 중단

soup = BeautifulSoup(res.text, 'html.parser')

# id=content 하위 네 번째 div (div[4])
content_div = soup.find(id="content")
divs = content_div.find_all("div", recursive=False)
if len(divs) >= 4:
    target_div = divs[3]  # Python index는 0-based, XPath는 1-based

    # 세 번째 dl 선택
    dls = target_div.find_all("dl", recursive=False)
    if len(dls) >= 3:
        target_dl = dls[2]

        dd = target_dl.find("dd")
        if dd:
            inner_div = dd.find("div")
            if inner_div:
                first_content = inner_div.find("div")
                if first_content:
                    print("📄 첫 줄 텍스트:", first_content.get_text(strip=True).splitlines()[0])
                else:
                    print("❌ <div> 내부 첫 번째 <div> 없음")
            else:
                print("❌ <dd> 내부 <div> 없음")
        else:
            print("❌ <dl> 내부 <dd> 없음")
    else:
        print("❌ dl[3] 없음")
else:
    print("❌ content 하위 div[4] 없음")

❌ content 하위 div[4] 없음


In [9]:
import requests
from bs4 import BeautifulSoup

res = requests.get(links[0], headers={'User-Agent': 'Mozilla/5.0'})
res.raise_for_status()

soup = BeautifulSoup(res.text, 'html.parser')

# div.wrap__text__body--info__text 요소를 먼저 확인
wrapper = soup.select_one('#content > div.wrap__text__body--info__text')
if not wrapper:
    print("❌ wrap__text__body--info__text not found (렌더링 안 됨)")
else:
    print("✅ wrapper found. Showing child <dl> elements:")
    for i, dl in enumerate(wrapper.find_all("dl")):
        print(f"\n--- DL #{i+1} ---")
        print(dl.get_text(strip=True))

❌ wrap__text__body--info__text not found (렌더링 안 됨)


In [12]:
pip install webdriver-manager

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# 크롬 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 브라우저 창을 띄우지 않음
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 접근할 URL
driver.get(links[0])
time.sleep(3)  # JavaScript 로딩 대기

# 본문 영역 추출 시도
try:
    element = driver.find_element(By.CSS_SELECTOR, '#content > div.wrap__text__body--info__text > dl:nth-child(3) > dd > div')
    print("✅ 본문 추출 결과:\n", element.text)
except Exception as e:
    print("❌ 요소 추출 실패:", e)

driver.quit()

✅ 본문 추출 결과:
 주제
晉나라 때의 백성들이 叔子를 더욱 그리워하였다는 내용의 부이다. 叔子는 羊祜의 字인데 훌륭한 인품으로 공적을 세워 명망이 높았던 관리이다. 그리고 자신의 후임으로 杜預라는 훌륭한 장수를 천거하여 아름다운 미담을 남겼다. 부의 대체적인 내용은 다음과 같다. 그대들은 杜預를 좋아하나 나는 양호를 사랑한다. 은택이 지금도 부로들에게 전해지고 있고, 특히 襄陽의 峴山에서 치적이 탁월하여 백성들이 세워 준 비석이 지금도 눈물짓게 한다. 양호는 공적도 훌륭하지만 어진 마음을 중시하여 누구나 흠모하여 감격하였는데, 특히 그의 仁德은 백성을 사랑할 때 더욱 빛났다. 양호는 가벼운 가죽옷으로 검소하게 지내면서 군사들을 잘 먹이니 적들의 마음도 우리에게 쏠렸고, 江水․漢水가 씻어주듯 가르치고 감싸주니 백성들은 남긴 옷을 볼 때마다 양호를 더욱 그리워하였다. 峴山의 정자에 양호와 두예를 기리고 있는데, 그 공적을 말하면 똑같다. 그런데 양양의 부로들이 눈물짓고, 남쪽의 오랑캐가 덕을 칭송하기로는 양호에게서 더욱 간절하다. 문필에도 뛰어났고 강물처럼 은택의 물결이 그침이 없으며 초목에까지 미쳐 지금도 칭송되니, 천년 후에도 영원할 것이다. 이제와 옛날을 회고하면 양호야말로 賢者라 이를 만하다. 이는 뚜렷한 실상이 있었기에 가능한 일이라, 천년 후에 무릎을 치며 성대한 글을 지어 탄식한다.

인물
李紀元(1830〜?)： 자는 春敷, 호는 省齋이다. 驪州李氏 23세로 求菴公 浚으로부터 10세, 李壽聃으로부터는 6세 후손이다. 成均進士로 文學과 孝友를 겸비하였다. 부인은 仁同 張氏 希旭의 따님이다.羊祜： 晉나라 泰山 南城사람. 자는 叔子, 荊州의 都督이 되었을 때, 학교를 열어 원근의 백성을 진무하여 인심을 크게 얻었다. 襄陽을 다스렸을 때에는 그의 공덕에 감격한 백성들이 峴山에 비석을 건립하였다. 검소하여 늘 가벼운 가죽 갖옷을 입었고, 이후 여러 벼슬을 지내며 공을 세웠으면서도 겸손한 마음을 지녀 내외의 명망이 높았다. 자신의 후임으로 훌륭한 장수인 杜預

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# CSV 불러오기
df = pd.read_csv("raw.csv")

# Selenium 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 결과 저장용
extracted_texts = []
error_log = []

# 크롤링 루프
for i, url in enumerate(df['webResourceURL']):
    print(f"[{i+1}/{len(df)}] 처리 중: {url}")

    if pd.isna(url) or not isinstance(url, str) or not url.startswith("http"):
        extracted_texts.append("")
        error_log.append({"index": i, "url": url, "error": "invalid or missing URL"})
        continue

    try:
        driver.get(url)
        time.sleep(3)
        element = driver.find_element(By.CSS_SELECTOR, '#content > div.wrap__text__body--info__text > dl:nth-child(3) > dd > div')
        extracted_texts.append(element.text.strip())
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        extracted_texts.append("")
        error_log.append({"index": i, "url": url, "error": str(e)})

# 드라이버 종료
driver.quit()

# 본문 열 추가
df['본문'] = extracted_texts

# 결과 저장
df.to_csv("시권_본문_추출결과.csv", index=False, encoding='utf-8-sig')

# 에러 로그 저장
error_df = pd.DataFrame(error_log)
error_df.to_csv("시권_크롤링_에러로그.csv", index=False, encoding='utf-8-sig')

print("\n✅ 본문 추출 완료 및 에러 로그 저장!")

/Users/songhune/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[1/513] 처리 중: http://www.emuseum.go.kr/detail?relicId=PS0100200100106199100000
❌ 오류 발생: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#content > div.wrap__text__body--info__text > dl:nth-child(3) > dd > div"}
  (Session info: chrome=136.0.7103.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001054983e0 cxxbridge1$str$ptr + 2829900
1   chromedriver                        0x00000001054906a8 cxxbridge1$str$ptr + 2797844
2   chromedriver                        0x0000000104fcdfbc cxxbridge1$string$len + 90140
3   chromedriver                        0x00000001050151bc cxxbridge1$string$len + 381468
4   chromedriver                        0x0000000105056044 cxxbridge1$string$len + 647332
5   chromedriver                        0x00000001050093f8 cxxbridge1$string$len + 332888
6   chromedrive